In [1]:
from Mollification import *

import trimesh
import os
import time

import meshio as mio

from buildGlueMap import *

root_folder = os.getcwd()

In [11]:
G = np.array([[[-1, -1], [-1, -1], [1, 0]], [[0,2], [2, 0], [3, 0]], [[1, 1], [-1, -1], [-1, -1]], [[1, 2], [-1, -1], [-1, -1]]])
FL = np.zeros((4, 3))
#E2FL = np.array([(0, 0), (0, 1), (0, 2), (1, 1), (2, 2), (2, 1), (1, 2), (3, 1), (3, 2)])
E2FL = np.array([[0, 0], [0, 1], [0, 2], [1, 1], [2, 2], [2, 1], [1, 2]])
#LE = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
LE = np.array([0, 1, 2, 3, 4, 5, 6])

FL2E = np.array([[[0], [1], [2]], [[2], [3], [6]], [[3], [5], [4]], [[6], [7], [8]]])



In [12]:
FL[E2FL[:,0], E2FL[:,1]] = LE
print(FL)

[[0. 1. 2.]
 [0. 3. 6.]
 [0. 5. 4.]
 [0. 0. 0.]]


In [4]:
((G[E2FL[:,0], E2FL[:,1]][:,0] != -1) & (G[E2FL[:,0], E2FL[:,1]][:,1] != -1))

array([False, False,  True,  True, False, False,  True])

In [5]:
FS = (E2FL[:,0], E2FL[:,1])
print(FS)

(array([0, 0, 0, 1, 2, 2, 1]), array([0, 1, 2, 1, 2, 1, 2]))


In [6]:
FSP = G[FS][((G[FS][:,0] != -1) & (G[FS][:,1] != -1)), :]
FSP = (FSP[:, 0], FSP[:, 1])
print(FSP)

(array([1, 2, 3]), array([0, 0, 0]))


In [7]:
FL[FSP] = LE[FL2E[FSP]][:,0]
print(FL)

[[0. 1. 2.]
 [2. 3. 6.]
 [3. 5. 4.]
 [6. 0. 0.]]


In [2]:
mesh = mio.read("../data/symmcube.obj")
F = mesh.cells_dict["triangle"]
V = mesh.points

In [5]:
print(FL)

[[141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.  

In [5]:
print(FLG)

[[141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [141.42135624 200.         141.42135624]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.        ]
 [200.         141.42135624 141.42135624]
 [141.42135624 141.42135624 200.  

In [8]:
print(G)

[[[17  2]
  [ 6  0]
  [14  1]]

 [[15  2]
  [ 7  0]
  [20  1]]

 [[21  2]
  [ 8  0]
  [18  1]]

 [[19  2]
  [ 9  0]
  [16  1]]

 [[13  2]
  [10  0]
  [15  0]]

 [[23  2]
  [11  0]
  [ 7  2]]

 [[ 0  1]
  [16  2]
  [23  0]]

 [[ 1  1]
  [14  2]
  [ 5  2]]

 [[ 2  1]
  [20  2]
  [11  2]]

 [[ 3  1]
  [18  2]
  [22  1]]

 [[ 4  1]
  [12  2]
  [17  0]]

 [[ 5  1]
  [22  2]
  [ 8  2]]

 [[13  1]
  [19  0]
  [10  1]]

 [[21  0]
  [12  0]
  [ 4  0]]

 [[15  1]
  [ 0  2]
  [ 7  1]]

 [[ 4  2]
  [14  0]
  [ 1  0]]

 [[17  1]
  [ 3  2]
  [ 6  1]]

 [[10  2]
  [16  0]
  [ 0  0]]

 [[19  1]
  [ 2  2]
  [ 9  1]]

 [[12  1]
  [18  0]
  [ 3  0]]

 [[21  1]
  [ 1  2]
  [ 8  1]]

 [[13  0]
  [20  0]
  [ 2  0]]

 [[23  1]
  [ 9  2]
  [11  1]]

 [[ 6  2]
  [22  0]
  [ 5  0]]]


In [3]:
delta = 1e-2
FL = igl.edge_lengths(V, F)

In [4]:
G = build_gluing_map(F)

end5 = time.time()

FLG, nMollG, nIterG = IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_MANHATTAN)
end6 = time.time()
print("Global optimization with Manhattan distance took: ", end6 - end5, " seconds.")

FLL, nMollL, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)

end7 = time.time()
print("Local optimization with one-by-one interpolation took: ", end7 - end6, " seconds.")

FLC, nMollC, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end8 = time.time()
print("Constant epsilon took: ", end8 - end7, " seconds.")


interior edge:  (0, 0) (6, 2) 0
interior edge:  (0, 1) (14, 0) 1
interior edge:  (0, 2) (17, 1) 2
interior edge:  (1, 0) (7, 2) 3
interior edge:  (1, 1) (20, 0) 4
interior edge:  (1, 2) (15, 1) 5
interior edge:  (2, 0) (8, 2) 6
interior edge:  (2, 1) (18, 0) 7
interior edge:  (2, 2) (21, 1) 8
interior edge:  (3, 0) (9, 2) 9
interior edge:  (3, 1) (16, 0) 10
interior edge:  (3, 2) (19, 1) 11
interior edge:  (4, 0) (10, 2) 12
interior edge:  (4, 1) (15, 2) 13
interior edge:  (4, 2) (13, 1) 14
interior edge:  (5, 0) (11, 2) 15
interior edge:  (5, 1) (7, 1) 16
interior edge:  (5, 2) (23, 1) 17
interior edge:  (6, 0) (16, 1) 18
interior edge:  (6, 1) (23, 2) 19
interior edge:  (7, 0) (14, 1) 20
interior edge:  (8, 0) (20, 1) 21
interior edge:  (8, 1) (11, 1) 22
interior edge:  (9, 0) (18, 1) 23
interior edge:  (9, 1) (22, 0) 24
interior edge:  (10, 0) (12, 1) 25
interior edge:  (10, 1) (17, 2) 26
interior edge:  (11, 0) (22, 1) 27
interior edge:  (12, 0) (19, 2) 28
interior edge:  (12, 2) (

In [6]:
np.linalg.norm(FLG - FL) / np.linalg.norm(FL)

0.0

In [7]:
np.linalg.norm(FLL - FL)/ np.linalg.norm(FL)

0.0

In [8]:
np.linalg.norm(FLC - FL) / np.linalg.norm(FL)

0.0